In [1]:
###########################################################################################
#
#   Load Libraries
#
###########################################################################################

# IPython utilities
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, HTML

# Standard libraries
import os
import session_info
from pyhere import here
from pathlib import Path

# Project specific
import gzip
import pandas as pd
from typing import List, Tuple
import ast

# Numerical and data handling
import numpy as np
import pandas as pd

#########################################################################################
#
#   Set Screen and Display Attributes
#
########################################################################################

# Display all outputs from a cell (not just the last one)
InteractiveShell.ast_node_interactivity = "all"

# Set notebook container width to full
display(HTML("<style>.container { width:100% !important; }</style>"))

# Set output window height to allow scrolling of long outputs
display(HTML("<style>div.output_scroll { height: 160em; }</style>"))

# Set maximum rows, columns, and display width
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
print(here())

/gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking


In [5]:
# """
# Skeleton script: compute global ancestry proportions per sample from per-chromosome
# local-ancestry VCF-like files where each sample field is encoded as ancestry counts
# separated by '|', e.g. for ancestries [CEU,YRI] a field looks like "0|2".

# Features:
# - Reads gzipped VCF files
# - Parses header to get sample names and (optionally) ancestry labels from '##Ancestries'
# - Two weighting modes: site-count (default) or interval-weighted (--weight)
# - Basic CLI with argparse

# Notes / assumptions:
# - Each sample column contains a single token (or FORMAT-like token where the first
#   subfield is the ancestry string). We take `field.split(':')[0]` to be safe.
# - The ancestry token is a '|' separated list of integers, one per ancestry.
# - The script works per input file (e.g. one chromosome file); you can pass multiple
#   files and results will be summed across them.
# - This is a skeleton; for very large datasets use cyvcf2/pysam and more memory-efficient
#   approaches.

# Requirements: Python 3.8+, standard library only.
# """

# def open_vcf_gz(path: str):
#     if path.endswith('.gz'):
#         return gzip.open(path, 'rt')
#     return open(path, 'r')


# def parse_header_and_ancestries(f) -> Tuple[List[str], List[str]]:
#     """Read header lines and return (sample_names, ancestries).
#     Assumes header lines start with '##' and the column header starts with '#CHROM'.
#     If a '##Ancestries:' line exists we try to parse it (JSON-like). If not found
#     caller must provide ancestries via CLI.
#     """
#     ancestries = None
#     sample_names = None
#     # we will read from the beginning; if f is a file object already at start
#     for line in f:
#         if line.startswith('##'):
#             if 'Ancestries' in line:
#                 # try to extract a simple JSON-like list: ["CEU","YRI"]
#                 try:
#                     import ast
#                     right = line.split(':', 1)[1].strip()
#                     ancestries = ast.literal_eval(right)
#                 except Exception:
#                     # leave ancestries as None and continue
#                     ancestries = None
#             continue
#         if line.startswith('#CHROM'):
#             parts = line.strip().split('\t')
#             sample_names = parts[9:]
#             break
#     return sample_names, ancestries


# def parse_ancestry_token(token: str) -> List[int]:
#     """Parse a sample token like '0|2' or '0|2:otherinfo' -> [0,2]
#     Returns list of ints (one per ancestry)."""
#     if token is None or token == '.':
#         return []
#     token = token.split(':', 1)[0]
#     parts = token.split('|')
#     try:
#         return [int(x) for x in parts]
#     except ValueError:
#         # if parsing fails, return empty to indicate missing data
#         return []


# def process_file(path: str,
#                  ancestries: List[str],
#                  weight_intervals: bool,
#                  sample_sums: dict,
#                  sample_total_weight: dict):
#     """Process one input file and update sample_sums (dict of sample->list(counts))
#     and sample_total_weight (dict of sample->total_weight) in-place.

#     If weight_intervals is False we just count sites (each site contributes its
#     ancestry counts directly). If True, we weight ancestry calls by genomic
#     interval to the next observed site (simple streaming approach).
#     """
#     num_ances = len(ancestries)

#     with open_maybe_gz(path) as fh:
#         # parse header
#         sample_names, file_ancestries = parse_header_and_ancestries(fh)
#         if sample_names is None:
#             raise RuntimeError(f"No #CHROM header found in {path}")
#         if file_ancestries:
#             # If file encodes ancestry labels, prefer them (but we still accept
#             # user-provided ancestries)
#             if file_ancestries and file_ancestries != ancestries:
#                 print("Warning: file ancestries differ from provided ancestries; using provided list.", file=sys.stderr)

#         # initialize per-sample running variables if not already present
#         for s in sample_names:
#             if s not in sample_sums:
#                 sample_sums[s] = [0.0] * num_ances
#                 sample_total_weight[s] = 0.0

#         prev_pos = None
#         # store previous per-sample counts for interval weighting
#         prev_counts = {s: None for s in sample_names}

#         # iterate lines from current file pointer
#         for line in fh:
#             if line.startswith('#'):
#                 continue
#             parts = line.rstrip('\n').split('\t')
#             chrom = parts[0]
#             pos = int(parts[1])
#             sample_fields = parts[9:]

#             # parse ancestry tokens for this site for all samples
#             parsed = [parse_ancestry_token(tok) for tok in sample_fields]

#             # ensure parsed lengths match num_ances for each sample (pad with zeros)
#             for i, arr in enumerate(parsed):
#                 if not arr:
#                     # missing -> treat as zeros
#                     parsed[i] = [0] * num_ances
#                 elif len(arr) < num_ances:
#                     parsed[i] = arr + [0] * (num_ances - len(arr))

#             if not weight_intervals:
#                 # simple site-count: add counts directly, weight=1 per site
#                 for sname, counts in zip(sample_names, parsed):
#                     # each value in counts represents number of alleles for that ancestry
#                     # so add them directly
#                     for a in range(num_ances):
#                         sample_sums[sname][a] += counts[a]
#                     sample_total_weight[sname] += 2.0  # two alleles per site
#                 continue

#             # if weighting by intervals: if this is the first site, just store counts
#             if prev_pos is None:
#                 prev_pos = pos
#                 for sname, counts in zip(sample_names, parsed):
#                     prev_counts[sname] = counts
#                 continue

#             # interval length between prev_pos and current pos
#             interval_len = pos - prev_pos
#             if interval_len < 0:
#                 interval_len = 0

#             # add prev_counts weighted by interval_len
#             for sname in sample_names:
#                 counts = prev_counts[sname]
#                 if counts is None:
#                     continue
#                 for a in range(num_ances):
#                     sample_sums[sname][a] += counts[a] * interval_len
#                 sample_total_weight[sname] += 2.0 * interval_len

#             # shift: current becomes previous
#             prev_pos = pos
#             for sname, counts in zip(sample_names, parsed):
#                 prev_counts[sname] = counts

#         # after loop: we don't add an interval for the final site (alternatively
#         # could extend to chromosome end if provided)


# def compute_proportions(sample_sums: dict, sample_total_weight: dict, ancestries: List[str]):
#     """Return a dict sample -> dict(ancestry->proportion)"""
#     out = {}
#     for s, sums in sample_sums.items():
#         total = sample_total_weight.get(s, 0.0)
#         # fall back to counting-mode denominator if weighting wasn't used
#         if total == 0.0:
#             # assume sums were raw allele counts over sites: total alleles = sum(sums)
#             total = sum(sums)
#         props = {}
#         for a, anc in enumerate(ancestries):
#             props[anc] = (sums[a] / total) if total > 0 else 0.0
#         out[s] = props
#     return out


# def write_tsv(output_path: str, proportions: dict, ancestries: List[str]):
#     with open(output_path, 'w') as out:
#         header = ['Sample'] + ancestries
#         out.write('\t'.join(header) + '\n')
#         for s in sorted(proportions.keys()):
#             row = [s] + [f"{proportions[s][a]:.6f}" for a in ancestries]
#             out.write('\t'.join(row) + '\n')


# def main():
#     p = argparse.ArgumentParser(description='Compute global ancestry proportions from local ancestry VCF-like files')
#     p.add_argument('inputs', nargs='+', help='Input per-chromosome files (vcf or vcf.gz).')
#     p.add_argument('--ancestries', nargs='+', help='Ordered list of ancestry labels (e.g. CEU YRI). If omitted, tries to read from file header.')
#     p.add_argument('--weight', action='store_true', help='Weight calls by interval (pos[i+1]-pos[i]) instead of simple site counts.')
#     p.add_argument('--out', default='global_ancestry.tsv', help='Output TSV file')
#     args = p.parse_args()

#     # prepare containers
#     sample_sums = {}          # sample -> [sum_per_ancestry]
#     sample_total_weight = {}  # sample -> total_weight (allele-units)

#     # determine ancestries: prefer command-line, else try to read from first file
#     ancestries = args.ancestries
#     if ancestries is None:
#         # try first file
#         with open_maybe_gz(args.inputs[0]) as fh:
#             _, file_ancestries = parse_header_and_ancestries(fh)
#             if file_ancestries is None:
#                 p.error('No ancestries provided and none found in file header. Pass --ancestries.')
#             ancestries = file_ancestries

#     print(f"Using ancestries: {ancestries}", file=sys.stderr)

#     # process each file
#     for path in args.inputs:
#         print(f"Processing {path} ...", file=sys.stderr)
#         process_file(path, ancestries, args.weight, sample_sums, sample_total_weight)

#     # compute proportions
#     proportions = compute_proportions(sample_sums, sample_total_weight, ancestries)

#     # write output
#     write_tsv(args.out, proportions, ancestries)
#     print(f"Wrote global ancestry table to {args.out}", file=sys.stderr)


# if __name__ == '__main__':
#     main()

In [23]:
def open_vcf_gz(path: str):
    """Open a plain text or gzipped file for reading."""
    return gzip.open(path, 'rt') if str(path).endswith('.gz') else open(path, 'r')

def parse_header_and_ancestries(path) -> Tuple[List[str], List[str]]:
    """Extract sample names and ancestries from the VCF header."""
    ancestries = None
    sample_names = None
    with open_maybe_gz(path) as f:
        for line in f:
            if line.startswith('##'):
                if 'Ancestries' in line:
                    try:
                        ancestries = ast.literal_eval(line.split(':', 1)[1].strip())
                    except Exception:
                        ancestries = None
                continue
            if line.startswith('#CHROM'):
                parts = line.strip().split('\t')
                sample_names = parts[9:]
                break
    return sample_names, ancestries

def load_vcf_to_df(path: str) -> pd.DataFrame:
    """Load VCF-like file into a pandas DataFrame (metadata skipped)."""
    with open_maybe_gz(path) as fh:
        header_line = None
        for line in fh:
            if line.startswith('#CHROM'):
                header_line = line.strip().split('\t')
                break
        if header_line is None:
            raise RuntimeError(f"No #CHROM header found in {path}")
        df = pd.read_csv(fh, sep='\t', header=None, names=header_line)
    return df

# Path to folder
folder_path = Path(here("input/simulations/two_populations/ground_truth/_m"))
files = sorted(folder_path.glob("*.vcf.gz"))

# Raise clear error if no files
if not files:
    raise FileNotFoundError(f"No .vcf.gz files found in: {folder_path.resolve()}")

# Inspect files
for vcf_path in files:
    print(f"\n=== Inspecting: {vcf_path.name} ===")
    samples, ancestries = parse_header_and_ancestries(vcf_path)
    print("Samples:", samples)
    print("Ancestries:", ancestries)
    
    df = load_vcf_to_df(vcf_path)
    print(df.head(), "\n")  # prints first few rows



=== Inspecting: chr1.vcf.gz ===
Samples: ['Sample_8', 'Sample_9', 'Sample_10', 'Sample_11', 'Sample_12', 'Sample_13', 'Sample_14', 'Sample_15', 'Sample_16', 'Sample_17', 'Sample_18', 'Sample_19', 'Sample_20', 'Sample_21', 'Sample_22', 'Sample_23', 'Sample_24', 'Sample_25', 'Sample_26', 'Sample_27', 'Sample_28', 'Sample_29', 'Sample_30', 'Sample_31', 'Sample_32', 'Sample_33', 'Sample_34', 'Sample_35', 'Sample_36', 'Sample_37', 'Sample_38', 'Sample_39', 'Sample_40', 'Sample_41', 'Sample_42', 'Sample_43', 'Sample_44', 'Sample_45', 'Sample_46', 'Sample_47', 'Sample_48', 'Sample_49', 'Sample_50', 'Sample_51', 'Sample_52', 'Sample_53', 'Sample_54', 'Sample_55', 'Sample_56', 'Sample_57', 'Sample_58', 'Sample_59', 'Sample_60', 'Sample_61', 'Sample_62', 'Sample_63', 'Sample_64', 'Sample_65', 'Sample_66', 'Sample_67', 'Sample_68', 'Sample_69', 'Sample_70', 'Sample_71', 'Sample_72', 'Sample_73', 'Sample_74', 'Sample_75', 'Sample_76', 'Sample_77', 'Sample_78', 'Sample_79', 'Sample_80', 'Sample_81

KeyboardInterrupt: 